In [15]:
from bs4 import BeautifulSoup
import requests #to load the page 
import csv
import time
from dateutil import parser
from tqdm import tqdm #good to have

import selenium
from selenium import webdriver ## webdriver is the tool to interact with the webpage 



In [16]:
def scrape_single_page(reviews, output, name):
    for review in reviews: 
        date = review.find('span', attrs={'class':'css-chan6m'}).string
        content = review.find('span', attrs={'class':'raw_09f24_T4Ezm'}).text.strip()
        raw = review.find('span',attrs={'class':'raw_09f24_T4Ezm'}).get_text
        temp = []
        temp.append(name)
        temp.append(date)
        temp.append(content)
        temp.append(raw)
        votes = review.find_all('span', attrs={'class':'css-12i50in'})
        for vote in votes:
            count = vote.find('span', attrs={'class':'css-1lr1m88'})
            if count: 
                temp.append(count.text)
            else: 
                temp.append('0')
                
        ## fails if there are revised reviews
        rating = review.find('div', attrs={'class':'star-rating'#change this accordingly})
        if rating: 
            temp.append('5')
        rating = review.find('div', attrs={'class':'star-rating'#change this accordingly})
        if rating: 
            temp.append('4')
        rating = review.find('div', attrs={'class':'star-rating'#change this accordingly})
        if rating: 
            temp.append('3')
        rating = review.find('div', attrs={'class':'star-rating'#change this accordingly})
        if rating: 
            temp.append('2')
        rating = review.find('div', attrs={'class':'star-rating'#change this accordingly})
        if rating: 
            temp.append('1')
        output.append(temp)

header = ['name','date','content','raw','useful','funny','cool','rating']

SyntaxError: invalid syntax (451893723.py, line 21)

In [ ]:
file = open('restaurant_list.csv', encoding = 'utf_8_sig')
csvreader = csv.reader(file)
restaurant_list = []
next(csvreader) 
for row in csvreader: 
    url = row[1]
    restaurant_list.append(url)
file.close()

for url in restaurant_list: 
    output = []
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')  
    
    #scrape the name of the restaurant 
    name = page.find('h1', attrs = {'class': 'css-12dgwvn'}).text #inspect html 
    reviews = page.find_all('div', attrs = {'class': 'css-12dgwvn'}) #inspect html 
    scrape_single_page(reviews, output, name)
    
    num_reviews = page.find('span', attrs = {'class': 'css-12dgwvn'}).string
    num_reviews = int(num_reviews.split('')[0])
    page_section = page.find('div', attrs = {'class': 'css-12dgwvn'})
    num_pages = page_section.find('span', attrs = {'class': 'css-12dgwvn'}).text
    num_pages = int(num_pages.split('')[-1]) - 1 
    
    url_list = []
    for i in range(10, num_pages*10, 10):
        url_list.append(url + '?start=' + str(i))
        
        
    print(name + ' ; ' + str(num_pages) + ' ; ' + strlen((url_list)) + ' ; ' + str(num_reviews))
    time.sleep(1)
    
    for new_url in tqdm(url_list): 
        num_reviews = 0
        sleep_time = 10
        sleep_num = 0
        
        while num_reviews == 0: 
            time.sleep(sleep_time)
            sleep_num += 1
            response = requests.get(new_url)
            page = BeautifulSoup(response.text, 'html_parser')
            reviews = page.find_all('div', attrs = {'class': '.....'}) #inspect html
            num_reviews = len(reviews)
            if sleep_num > 2: 
                print("sleep more than 3 for: " + new_url)
        scrape_single_page(reviews, output, name)
        
    cur_size = len(output)
    print('newly added: ' + str(cur_size))
    
    file name = name + '.csv'

FileNotFoundError: [Errno 2] No such file or directory: 'restaurant_list.csv'

In [24]:
#Spotify imports
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json


In [86]:
#Initializing API calls using Spotify tokens'

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token 

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}


# here we have obtained the token we can use to get the attributes we want from our songs
token = get_token()

In [91]:
#search for the track on spotify api
def search_for_track(token, track_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={track_name}&type=track&limit=1"

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    if len(json_result) == 0:
        print("song not found")
        return None
    return json_result

#slice the track call to get track id
def get_track_id(track_dict):
    return track_dict['tracks']['items'][0]['id']

#extract track features 
def get_track_features(token, track_id):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}?country=US"
    headers = get_auth_header(token)
    #query = f"?{track_id}"

    query_url = url #+ query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)
    return json_result

#print(search_for_track(token,"ghost")['tracks']['items'][0]['id'])
#print(type(get_track_features(token, "6I3mqTwhRpn34SLVafSH7G")))

track = search_for_track(token, 'Last Christmas')
track_id = get_track_id(track)
features = get_track_features(token, track_id)

print(features)



{'danceability': 0.735, 'energy': 0.478, 'key': 2, 'loudness': -12.472, 'mode': 1, 'speechiness': 0.0293, 'acousticness': 0.189, 'instrumentalness': 2.48e-06, 'liveness': 0.355, 'valence': 0.947, 'tempo': 107.682, 'type': 'audio_features', 'id': '2FRnf9qhLbvw8fu4IBXx78', 'uri': 'spotify:track:2FRnf9qhLbvw8fu4IBXx78', 'track_href': 'https://api.spotify.com/v1/tracks/2FRnf9qhLbvw8fu4IBXx78', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2FRnf9qhLbvw8fu4IBXx78', 'duration_ms': 262960, 'time_signature': 4}
